In [229]:
import trimesh
import pymeshlab
import numpy as np
import open3d as o3d
from numpy import linalg as LA
from functools import reduce

In [231]:
ms = pymeshlab.MeshSet()
ms.load_new_mesh("C:\\Users\\adamt\\open3d_data\\download\\drill\\data\\drill_1.6mm_30_cyb.ply")

# Get the current mesh
mesh = ms.current_mesh()

# Access vertex coordinates as NumPy array
vertex_array = mesh.vertex_matrix()

print(vertex_array)
vertex_array[0]

[[-0.004688    0.09172909  0.02415076]
 [-0.004102    0.09172909  0.02415076]
 [-0.003809    0.09175673  0.02453249]
 ...
 [-0.006153    0.15663855  0.02149424]
 [-0.00586     0.15663855  0.02149424]
 [-0.005567    0.15662847  0.02098651]]


array([-0.004688  ,  0.09172909,  0.02415076])

In [233]:
def centroid_pcd(P):
    cen = np.zeros(3)
    K = np.size(P)/3
    for xyz in P:
        cen = np.add(cen,xyz)
    return np.multiply(cen,1/K)
cen = centroid_pcd(vertex_array)

In [235]:
pcd = o3d.io.read_point_cloud("C:\\Users\\adamt\\open3d_data\\download\\drill\\data\\drill_1.6mm_30_cyb.ply")
centroid = o3d.geometry.PointCloud()
cen = centroid_pcd(vertex_array)
centroid.points = o3d.utility.Vector3dVector([cen])
centroid.colors = o3d.utility.Vector3dVector([[0,1,0]])
sphere = o3d.geometry.TriangleMesh.create_sphere(radius=0.001)  # Adjust radius for size
sphere.translate(cen)  # Move sphere to the point's location
sphere.paint_uniform_color([1, 0, 0])  # Set color (red)
o3d.visualization.draw_geometries([sphere,pcd])

In [237]:
def covar(P):
    P_copy = P.copy()
    cen = centroid_pcd(P_copy)
    K = np.size(P_copy)/3
    for i in range(int(K)):
        P_copy[i] = P_copy[i] - cen
    return np.multiply(1/K,np.dot(P_copy.transpose(),P_copy))
print(covar(vertex_array))

[[ 4.23497832e-05 -1.90084358e-05 -3.18871790e-06]
 [-1.90084358e-05  4.48096801e-04 -3.69493172e-05]
 [-3.18871790e-06 -3.69493172e-05  1.56421453e-05]]


In [239]:
def mod_covar(P,R:float,p):
    v = []
    for i in range(int(np.size(P)/3)):
        v.append(R - LA.norm(P[i] - p))
    scalar = 1/(reduce(lambda x,y: x + y, v,0))
    M = sum(np.multiply(v[i],np.matmul(np.transpose([P[i] - p]),[P[i]-p]))
                 for i in range(int(np.size(P)/3)))
    return np.multiply(scalar, M)
mod_covar(vertex_array,0.001,vertex_array[200])

array([[ 1.69859894e-04,  3.85447679e-04, -1.25703179e-05],
       [ 3.85447679e-04,  1.83482135e-03, -5.49568632e-05],
       [-1.25703179e-05, -5.49568632e-05,  1.54036375e-05]])

In [249]:
#Get eigenvectors + values
def EVD(P,R,p):
    eigenvalues, eigenvectors = LA.eig(mod_covar(vertex_array,R,p))
    eig = eigenvalues.copy()
    first = np.where(eigenvalues == max(eig))[0][0]
    eig = np.delete(eig,first)
    second = np.where(eigenvalues == max(eig))[0][0]
    eig = np.delete(eig,np.where(eig == max(eig)))
    third = np.where(eigenvalues == eig[0])[0][0]
    sorted_eigenvalues = np.array([eigenvalues[first],eigenvalues[second],eigenvalues[third]])
    sorted_eigenvectors = np.column_stack((eigenvectors[:,first],eigenvectors[:,second],eigenvectors[:,third]))
    return sorted_eigenvalues,sorted_eigenvectors
    
def original_EVD(P,R,p):
    return LA.eig(mod_covar(vertex_array,R,p))

def dis_amb(P,R,p):
    eigenvalues, eigenvectors = EVD(P,R,p)
    x_pos = eigenvectors[:,0]
    y_pos = eigenvectors[:,1]
    z_pos = eigenvectors[:,2]
    x_neg = np.multiply(-1,x_pos)
    y_neg = np.multiply(-1,y_pos)
    z_neg = np.multiply(-1,z_pos)
    for i in  range(int(np.size(P)/3)):
        Sx_pos = 0
        Sx_neg = 0
        if np.dot((P[i] - p),x_pos) >= 0:
            Sx_pos = Sx_pos + 1
        else:
            Sx_neg = Sx_neg + 1
    if Sx_pos >= Sx_neg:
        x = x_pos
    else:
        x = x_neg
    for i in range(int(np.size(P)/3)):
        Sz_pos = 0
        Sz_neg = 0
        if np.dot((P[i] - p),z_pos) >= 0:
            Sz_pos = Sz_pos + 1
        else:
            Sz_neg = Sz_neg + 1
    if Sz_pos >= Sz_neg:
        z = z_pos
    else:
        z = z_neg
    y = np.cross(z,x)
    return x,y,z
#SORT THE EIGENVALUE + VECTORS;
#EVD(vertex_array,0.001,vertex_array[20])
#original_EVD(vertex_array,0.001,vertex_array[20])
dis_amb(vertex_array,0.001,vertex_array[20])

(array([-0.00204177,  0.99297526, -0.11830455]),
 array([-0.96906716,  0.02723163,  0.24529019]),
 array([0.24678871, 0.11514588, 0.96220411]))

In [243]:
def draw_geometries(pcds):
    """
    Draw Geometries
    Args:
        - pcds (): [pcd1,pcd2,...]
    """
    o3d.visualization.draw_geometries(pcds)

def get_o3d_FOR(origin=[0, 0, 0],size=0.05):
    """ 
    Create a FOR that can be added to the open3d point cloud
    """
    mesh_frame = o3d.geometry.TriangleMesh.create_coordinate_frame(
    size=size)
    mesh_frame.translate(origin)
    return(mesh_frame)

def vector_magnitude(vec):
    """
    Calculates a vector's magnitude.
    Args:
        - vec (): 
    """
    magnitude = np.sqrt(np.sum(vec**2))
    return(magnitude)


def calculate_zy_rotation_for_arrow(vec):
    """
    Calculates the rotations required to go from the vector vec to the 
    z axis vector of the original FOR. The first rotation that is 
    calculated is over the z axis. This will leave the vector vec on the
    XZ plane. Then, the rotation over the y axis. 

    Returns the angles of rotation over axis z and y required to
    get the vector vec into the same orientation as axis z
    of the original FOR

    Args:
        - vec (): 
    """
    # Rotation over z axis of the FOR
    gamma = np.arctan(vec[1]/vec[0])
    Rz = np.array([[np.cos(gamma),-np.sin(gamma),0],
                   [np.sin(gamma),np.cos(gamma),0],
                   [0,0,1]])
    # Rotate vec to calculate next rotation
    vec = Rz.T@vec.reshape(-1,1)
    vec = vec.reshape(-1)
    # Rotation over y axis of the FOR
    beta = np.arctan(vec[0]/vec[2])
    Ry = np.array([[np.cos(beta),0,np.sin(beta)],
                   [0,1,0],
                   [-np.sin(beta),0,np.cos(beta)]])
    return(Rz, Ry)

def create_arrow(scale=0.001):
    """
    Create an arrow in for Open3D
    """
    cone_height = scale*0.002
    cylinder_height = scale*0.008
    cone_radius = scale/10
    cylinder_radius = scale/20
    mesh_frame = o3d.geometry.TriangleMesh.create_arrow(cone_radius=0.001,
        cone_height=cone_height,
        cylinder_radius=0.0005,
        cylinder_height=cylinder_height)
    return(mesh_frame)

def get_arrow(origin=[0, 0, 0], end=None, vec=None):
    """
    Creates an arrow from an origin point to an end point,
    or create an arrow from a vector vec starting from origin.
    Args:
        - end (): End point. [x,y,z]
        - vec (): Vector. [i,j,k]
    """
    scale = 0.001
    Ry = Rz = np.eye(3)
    T = np.array([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1]])
    T[:3, -1] = origin
    if end is not None:
        vec = np.array(end) - np.array(origin)
    elif vec is not None:
        vec = np.array(vec)
    if end is not None or vec is not None:
        scale = vector_magnitude(vec)
        Rz, Ry = calculate_zy_rotation_for_arrow(vec)
    mesh = create_arrow(scale)
    # Create the arrow
    mesh.rotate(Ry, center=np.array([0, 0, 0]))
    mesh.rotate(Rz, center=np.array([0, 0, 0]))
    mesh.translate(origin)
    return(mesh)


# Create a Cartesian Frame of Reference

# Create an arrow from point (5,5,5) to point (10,10,10)
# arrow = get_arrow([5,5,5],[10,10,10])

# Create an arrow representing vector vec, starting at (5,5,5)
#arrow = get_arrow([5,5,5],vec=[5,5,5])

# Create an arrow in the same place as the z axis
#arrow = get_arrow()

In [245]:
def get_discriptors(P,R,p,pcd,cen):
    FOR = get_o3d_FOR()
    x,y,z = dis_amb(vertex_array,R,p)
    arrow_x = get_arrow(p,vec=x)
    arrow_y = get_arrow(p,vec=y)
    arrow_z = get_arrow(p,vec=z)
    # Draw everything
    draw_geometries([FOR,arrow_x,arrow_y,arrow_z,pcd,sphere])
get_discriptors(vertex_array,0.001,vertex_array[320],pcd,sphere)

In [209]:
def normalize(v:list):
    return np.multiply(1/np.linalg.norm(v),v)

In [338]:
def sort_neighborhood(P,R,p):
    copy = P.copy()
    k = 0
    for i in range(int(np.size(P)/3)):
        if np.linalg.norm(P[i] - p) > R:
            copy = np.delete(copy,i-k,axis=0)
            k = k + 1
    return copy
sort_neighborhood(vertex_array,0.001,vertex_array[0])

array([[-0.004688  ,  0.09172909,  0.02415076],
       [-0.004102  ,  0.09172909,  0.02415076],
       [-0.003809  ,  0.09175673,  0.02453249]])

In [347]:
def normals(P,R):
    n = []
    for xyz in P:
        neighborhood = sort_neighborhood(P,R,xyz)
        eigval, eigvec = EVD(neighborhood,R,xyz)
        n.append(eigvec[:,2])
    return n
normals(vertex_array,0.001)

[0.17632453 0.08606708 0.98056214]
[0.16299209 0.08773905 0.98271839]
[0.15463344 0.09572084 0.98332397]
[0.14266295 0.1083764  0.98382002]
[0.13236004 0.1063041  0.98548478]
[0.11839736 0.11126922 0.98671233]
[0.10449736 0.11361843 0.98801374]
[0.09708501 0.10882861 0.98930826]
[0.09175802 0.10408631 0.99032646]
[0.07165378 0.11045051 0.99129532]
[0.06067933 0.10993524 0.99208481]
[0.07881411 0.09464516 0.99238633]
[0.06671706 0.09639568 0.99310458]
[0.05903128 0.09598412 0.9936309 ]
[0.10564815 0.07422713 0.99162937]
[0.07058887 0.08562652 0.99382358]
[0.06514937 0.08537049 0.994217  ]
[0.11173531 0.06757221 0.99143795]
[0.2163395  0.08428615 0.97267315]
[0.247212   0.1088902  0.96282353]
[-0.24678871 -0.11514588 -0.96220411]
[0.24086013 0.11910927 0.96322343]
[0.23334258 0.1230324  0.96457984]
[0.22110358 0.12448853 0.96727236]
[0.20790666 0.1257849  0.97002731]
[0.19382719 0.12690058 0.97279353]
[0.17506237 0.12040736 0.97716694]
[0.16016699 0.11615797 0.98023153]
[0.14705398 0.124

[array([0.17632453, 0.08606708, 0.98056214]),
 array([0.16299209, 0.08773905, 0.98271839]),
 array([0.15463344, 0.09572084, 0.98332397]),
 array([0.14266295, 0.1083764 , 0.98382002]),
 array([0.13236004, 0.1063041 , 0.98548478]),
 array([0.11839736, 0.11126922, 0.98671233]),
 array([0.10449736, 0.11361843, 0.98801374]),
 array([0.09708501, 0.10882861, 0.98930826]),
 array([0.09175802, 0.10408631, 0.99032646]),
 array([0.07165378, 0.11045051, 0.99129532]),
 array([0.06067933, 0.10993524, 0.99208481]),
 array([0.07881411, 0.09464516, 0.99238633]),
 array([0.06671706, 0.09639568, 0.99310458]),
 array([0.05903128, 0.09598412, 0.9936309 ]),
 array([0.10564815, 0.07422713, 0.99162937]),
 array([0.07058887, 0.08562652, 0.99382358]),
 array([0.06514937, 0.08537049, 0.994217  ]),
 array([0.11173531, 0.06757221, 0.99143795]),
 array([0.2163395 , 0.08428615, 0.97267315]),
 array([0.247212  , 0.1088902 , 0.96282353]),
 array([-0.24678871, -0.11514588, -0.96220411]),
 array([0.24086013, 0.11910927,